In [ ]:
!apt-get update

import os
import csv
from datetime import datetime
import re
import numpy as np
import pandas as pd

In [ ]:
#converting excel file to dataframe 
sales_df=pd.DataFrame(pd.read_excel("WalmartSales_1D4B9EA390.xlsx"))

To analyze the performance of sales and revenue at the city and branch level

In [ ]:
#calculate total revenue 
sales_df['Total_Revenue']=sales_df['Unit price']*sales_df['Quantity']


performance=sales_df.groupby(
    ['City','Branch']).agg(
        total_sales=('Invoice ID','count'),
        total_revenue=('Total_Revenue','sum'),
        avg_rating=('Rating','mean')
    ).reset_index().round(2)


#saving it to csv
performance.to_csv('/root/Desktop/sales/Output/sales_revenue_performance.csv',na_rep='NaN')

What is the average price of an item sold at each branch of the city

In [ ]:
branch_avg_price=sales_df.groupby(
    ['City','Branch']).agg(
    avg_price=('Unit price','mean')
).reset_index().round(2)


#saving it to csv
branch_avg_price.to_csv('/root/Desktop/sales/Output/branch_avg_price.csv',na_rep='NaN')

Analyze the performance of sales and revenue, Month over Month across the Product line, Gender, and Payment Method, and identify the focus areas to get better sales for April 2019.

In [ ]:
#converitng date format
sales_df['Sales_Date'] = pd.to_datetime(sales_df['Date'], format='%m/%d/%Y')

#extracting the month from the date
sales_df['Month'] = sales_df['Sales_Date'].dt.strftime('%m')

#removing unnecessary column
sales_df.drop(['Sales_Date'],axis=1,inplace=True)

#monthly sales
monthly_sales = (
    sales_df.groupby(['Month', 'Product line', 'Gender', 'Payment'])
    .agg(
        Total_Revenue=('Total_Revenue', 'sum'),
        Total_Sales=('Invoice ID', 'count'),
        Avg_Rating=('Rating', 'mean')
    )
    .reset_index()
    .sort_values(by=['Month', 'Product line', 'Gender', 'Payment'])
)


#previous month revrenue
monthly_sales['Previous_Month_Revenue']=monthly_sales['Total_Revenue'].shift(1)

#previous month sales
monthly_sales['Previous_Month_Sales']=monthly_sales['Total_Sales'].shift(1)

#change in revenue
monthly_sales['Revenue_Change']=monthly_sales['Total_Revenue']-monthly_sales['Previous_Month_Revenue']

#change in sales 
monthly_sales['Sales_Change']=monthly_sales['Total_Sales']-monthly_sales['Previous_Month_Sales']

#filtering data which has negative values
focus_areas=monthly_sales[((monthly_sales['Revenue_Change'] < 0) | (monthly_sales['Sales_Change']<0))].round(2)

#sorting results with most heavy declines
focus_areas=focus_areas.sort_values(by=['Revenue_Change','Sales_Change'])

#saving it to csv
focus_areas.to_csv('/root/Desktop/sales/Output/focus_areas_april_2019.csv',na_rep='NaN')